In [1]:
#code inspired from https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


from torch.autograd import Variable
## each image is 490 x 326 

#create the model classes

#how to initialize weights function
def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

#this is just a layer where you pass in a lambda and it does the rest
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)

#basic block with shortcut (not a bottleneck block)
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                top = (int) ((self.expansion*planes - in_planes) / 2)
                bot = (self.expansion*planes - in_planes) - top
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::stride, ::stride], (0, 0, 0, 0, top, bot), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )
      

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
##bottleneck block
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, option='B'):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            if option == 'A':
                top = (int) ((self.expansion*planes - in_planes) / 2)
                bot = (self.expansion*planes - in_planes) - top
                self.shortcut = LambdaLayer(lambda x:
                                                F.pad(x[:, :, ::stride, ::stride], (0, 0, 0, 0, top, bot), "constant", 0)) 
            elif option == 'B':
                    self.shortcut = nn.Sequential(
                        nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                        nn.BatchNorm2d(self.expansion*planes)
                    )
            

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2, option='B'):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, option=option)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=4, option=option)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=4, option=option)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, option=option)
        
        self.linear = nn.Linear(4096, num_classes)
        self._initialize_weights()

    def _make_layer(self, block, planes, num_blocks, stride, option):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, option))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

        

In [1]:
import visualize
from visualize import TestVisualizer
import load_data as ld
from load_data import PhoneDataset
from torch.utils.data import DataLoader

test_data = ld.get_data(labels_file='./data/labels/labels.txt',
                       root_dir='./data/train')

for i in range(len(test_data)): 
    img_name, img, target = test_data.get_labels(i)
    testv = TestVisualizer(img_name, [target[0, 0], target[0, 1]])
    #print(testv.get_error())
    testv.save_image('./meatheads/', drawbox=True)

In [2]:
def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

In [12]:
import load_data as ld
import torch
from load_data import PhoneDataset
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

#torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

train_data = ld.get_data(labels_file='./data/labels/labels.txt', 
                           root_dir='./data/train/',
                                        mode="absolute");

validation_data = ld.get_data(labels_file='./data/labels/labels.txt',
                                   root_dir='./data/validation/')

test_data = ld.get_data(labels_file='./data/labels/labels.txt',
                       root_dir='./data/test')

train_loader = DataLoader(dataset=train_data, batch_size=2, shuffle=False);
val_loader = DataLoader(dataset=validation_data, batch_size=2)

#for i_batch, (image, target) in enumerate(train_loader, 0):
#    print(target)

#image = torch.ones([3,326,490])
#mtransform(image)

for i in range(len(train_data)):
    sample_name, image, target = train_data.get_labels(i)
    image = image.float()
    newimage = torchvision.transforms.functional.normalize(image, (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    print(newimage == image)
    

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [

In [5]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


net = ResNet18().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(100):
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader, 0):
        net.train()
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs.float())
        loss = criterion(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2 == 1: #print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 2))
            running_loss = 0.0
    
    print("Validation loss: ")
    loss = 0.0
    
    for i, (inputs, labels) in enumerate(val_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        net.eval()
        with torch.no_grad():
            outputs = net(inputs.float())
            loss += criterion(outputs, labels.float()).item()
    
    print("Epoch Loss: {}".format(loss))
    print("------------------------------------------------------------")
    
print('Finished Training')

[1,     2] loss: 17.462
[1,     4] loss: 1.024
[1,     6] loss: 4.980
[1,     8] loss: 5.123
[1,    10] loss: 3.295
[1,    12] loss: 0.890
[1,    14] loss: 2.590
[1,    16] loss: 12.513
[1,    18] loss: 1.544
[1,    20] loss: 4.029
[1,    22] loss: 2.259
[1,    24] loss: 6.538
[1,    26] loss: 2.714
[1,    28] loss: 6.098
[1,    30] loss: 2.917
[1,    32] loss: 0.521
[1,    34] loss: 0.621
[1,    36] loss: 0.430
[1,    38] loss: 0.407
[1,    40] loss: 0.772
[1,    42] loss: 0.872
[1,    44] loss: 3.636
[1,    46] loss: 3.508
[1,    48] loss: 0.603
[1,    50] loss: 1.672
[1,    52] loss: 0.586
Validation loss: 
Epoch Loss: 384.62939527630806
------------------------------------------------------------
[2,     2] loss: 0.877
[2,     4] loss: 2.118
[2,     6] loss: 0.697
[2,     8] loss: 2.646
[2,    10] loss: 0.708
[2,    12] loss: 0.129
[2,    14] loss: 1.019
[2,    16] loss: 0.941
[2,    18] loss: 0.371
[2,    20] loss: 0.291
[2,    22] loss: 0.413
[2,    24] loss: 0.307
[2,    26] los

KeyboardInterrupt: 

In [1]:
import load_data as ld
from load_data import PhoneDataset
from torch.utils.data import DataLoader

test_data = ld.get_data(labels_file='./data/labels/labels.txt',
                       root_dir='./data/test/')

test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
print("HEY")

for i, (inputs, labels) in enumerate(test_loader, 0):
    print("YO")
    inputs, labels = inputs.to(device), labels.to(device)
    net.eval()
    with torch.no_grad():
        outputs = net(inputs.float())
        print("Net output:")
        print(outputs)
        print("Actual output:")
        print(labels.float())

YOHO
HEY
YO


NameError: name 'device' is not defined

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [1]:
from visualize import TestVisualizer

def test(args, model, device, test_loader):
    for data, target in test_loader:
        testv = TestVisualizer('./data/train/')


In [5]:
torch.cuda.empty_cache()

In [2]:
import torch

x = torch.Tensor(2, 1, 3)
print(x.size())
print(x.squeeze().size())

torch.Size([2, 1, 3])
torch.Size([2, 3])


In [5]:
!nvidia-smi

Sat Jan 18 23:18:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1060    Off  | 00000000:01:00.0  On |                  N/A |
| N/A   66C    P2    34W /  N/A |   2972MiB /  3011MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
import torch

def find(tensor, values):
    return torch.nonzero(tensor[..., None] == values)

ten = torch.zeros((2, 2, 2))
ten[0, 1, 1] = 1


print(ten)
res = find(ten, torch.FloatTensor([1]))

print(res)
mlist = [element.item() for element in res.flatten()][0:3]
print(mlist)

tensor([[[0., 0.],
         [0., 1.]],

        [[0., 0.],
         [0., 0.]]])
tensor([[0, 1, 1, 0]])
[0, 1, 1]


In [1]:
import torch
from skimage import io, transform
from PIL import Image

image = io.imread('./data/train/0.jpg')
image = torch.from_numpy(image.transpose((2,0,1)))
im = Image.open('./data/train/0.jpg')

print(image[2, 44, 409]) #35, 36, 38
print(im.getpixel((409, 44)))
im.putpixel((409, 44), (255, 255, 255))
print(im.getpixel((409, 44)))
print(image[2, 44, 409])


tensor(38, dtype=torch.uint8)
(35, 36, 38)
(255, 255, 255)
tensor(38, dtype=torch.uint8)


In [3]:
from load_data import PhoneDataset
import safetransforms as sft
import torch
from PIL import Image

import load_data as ld
from load_data import PhoneDataset, DatasetBuilder
from torch.utils.data import DataLoader

import visualize
from visualize import TestVisualizer

train_data = PhoneDataset(labels_file='./data/labels/labels.txt',
                       root_dir='./data/validation/')

mbuilder = DatasetBuilder(train_data, './generated/data/validation', PILtransforms=[sft.RandomFlip(0.9), sft.SafeRotate(0.9), sft.SafeCropRescale(0.9)], 
                         generate=30)

mbuilder.generate()


test_data = PhoneDataset(labels_file='./generated/labels.txt',
                       root_dir='./generated/train/')


for i in range(len(test_data)): 
    img_name, img, target = test_data.get_labels(i)
    testv = TestVisualizer(img_name, [target[0, 0], target[0, 1]])
    testv.save_image('./meatheads/', drawbox=True)


./generated/data/validation/111_0.jpg
./generated/data/validation/111_1.jpg
./generated/data/validation/111_2.jpg
./generated/data/validation/111_3.jpg
./generated/data/validation/111_4.jpg
./generated/data/validation/111_5.jpg
./generated/data/validation/111_6.jpg
./generated/data/validation/111_7.jpg
./generated/data/validation/111_8.jpg
./generated/data/validation/111_9.jpg
./generated/data/validation/111_10.jpg
./generated/data/validation/111_11.jpg
./generated/data/validation/111_12.jpg
./generated/data/validation/111_13.jpg
./generated/data/validation/111_14.jpg
./generated/data/validation/111_15.jpg
./generated/data/validation/111_16.jpg
./generated/data/validation/111_17.jpg
./generated/data/validation/111_18.jpg
./generated/data/validation/111_19.jpg
./generated/data/validation/111_20.jpg
./generated/data/validation/111_21.jpg
./generated/data/validation/111_22.jpg
./generated/data/validation/111_23.jpg
./generated/data/validation/111_24.jpg
./generated/data/validation/111_25.

./generated/data/validation/118_2.jpg
./generated/data/validation/118_3.jpg
./generated/data/validation/118_4.jpg
./generated/data/validation/118_5.jpg
./generated/data/validation/118_6.jpg
./generated/data/validation/118_7.jpg
./generated/data/validation/118_8.jpg
./generated/data/validation/118_9.jpg
./generated/data/validation/118_10.jpg
./generated/data/validation/118_11.jpg
./generated/data/validation/118_12.jpg
./generated/data/validation/118_13.jpg
./generated/data/validation/118_14.jpg
./generated/data/validation/118_15.jpg
./generated/data/validation/118_16.jpg
./generated/data/validation/118_17.jpg
./generated/data/validation/118_18.jpg
./generated/data/validation/118_19.jpg
./generated/data/validation/118_20.jpg
./generated/data/validation/118_21.jpg
./generated/data/validation/118_22.jpg
./generated/data/validation/118_23.jpg
./generated/data/validation/118_24.jpg
./generated/data/validation/118_25.jpg
./generated/data/validation/118_26.jpg
./generated/data/validation/118_2

In [10]:
train_data = PhoneDataset(labels_file='./data/labels/labels.txt',
                       root_dir='./data/validation/')

for i in range(len(train_data)): 
    img_name, img, target = train_data.get_labels(i)
    testv = TestVisualizer(img_name, [target[0, 0], target[0, 1]])
    testv.save_image('./meat_heads/', drawbox=True)
    print("DUN")
    
im = Image.open('./meatheads/intermed.jpg')
width, height = im.size

#pixel testing garbage
mymax = 0
idx = []
for i in range(width):
    for j in range(height):
        prev_max = mymax
        mymax = max(im.getpixel((i,j))[0], mymax)
        
        if mymax != prev_max:
            idx = [i,j]
        
        #print(im.getpixel((i,j)))
        
print(mymax)
print(idx)

DUN
DUN
DUN
DUN
DUN
DUN
DUN
DUN
DUN
DUN


In [7]:
import torch
import torchvision

def find(tensor, values):
    return torch.nonzero(tensor[..., None] == values)

ten = torch.zeros((3, 326, 490), dtype=torch.uint8)
ten[0, 0, 0] = 1
ids = find(ten,torch.FloatTensor([1]))
print(ids)

pil = transformed_label_image = torchvision.transforms.ToPILImage()(ten)
pil.putpixel((3,40), (255, 0, 0))
pil.putpixel((2,2), (255, 0, 0))

#print(pil.getpixel((0,0)))
#print(pil.getpixel((0,5)))
#print(pil.getextrema())
ten = torchvision.transforms.ToTensor()(pil)

idx = torch.max(ten, dim=2)

ids = find(ten, torch.FloatTensor([1]))

#indices = (ten[0] != 0).nonzero().tolist()
#mekek = torch.max(ten[0], dim=0)
m = ten[0].view(1, -1).argmax(1)
indices = torch.cat(((m / 490).view(-1, 1), (m % 490).view(-1, 1)), dim=1)

print(indices[0,1])


tensor([[0, 0, 0, 0]])
tensor(3)


In [3]:
import torch
import torchvision
import math
import PIL
import random
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms, utils
from torchvision.transforms import functional as func

def cropp_rotated(image, degrees):
    x, y = image.size
    cosA = abs(math.cos(math.radians(degrees)))
    sinA = abs(math.sin(math.radians(degrees)))

    a = x * cosA 
    b = x * sinA

    relation = a / (a + b)
    right_indent1 = a - x * relation * cosA

    relation = b / (a + b)
    bottom_ident1 = b - x * relation *sinA


    c = y * cosA 
    d = y * sinA

    relation = c / (c + d)
    right_indent2 = c - y * relation * cosA

    relation = d / (c + d)
    bottom_ident2 = d - y * relation *sinA

    right_indent = max(right_indent1, right_indent2)
    top_indent = max(bottom_ident1, bottom_ident2)

    #size of rotated image:
    w_rotated = x * cosA + y * sinA
    h_rotated = y * cosA + x * sinA


    box = [
    int(top_indent), 
    int(right_indent), 
    int(h_rotated - top_indent)+1,
    int(w_rotated - right_indent)
    ]
    
    return box


rotation = 270
im = Image.open('./data/train/0.jpg')
im = torchvision.transforms.functional.rotate(im, rotation, expand=True, resample=False)


#box = cropp_rotated(im, rotation)
#print(box)
#im = torchvision.transforms.functional.rotate(im, rotation, expand=True, resample=False)
#im = torchvision.transforms.functional.crop(im, box[0], box[1], box[2]-box[0], box[3]-box[1])
#print(im.size)
im.save('./meatheads/0.jpg')



In [32]:
import torch
import torchvision

def find(tensor, values):
    return torch.nonzero(tensor[..., None] == values)

ten = torch.zeros((3, 326, 490), dtype=torch.uint8)
ten[0, 0, 0] = 1
ids = find(ten,torch.FloatTensor([1]))
print(ids)

pil = transformed_label_image = torchvision.transforms.ToPILImage()(ten)
pil.putpixel((3,40), (255, 0, 0))
pil.putpixel((2,2), (255, 0, 0))

top, right, width, height = 0, 0, 50, 50
print(pil.getpixel((2,2)))

count = 0

for i in range(top, top+height):
    for j in range(right, right+width):
        if pil.getpixel((j, i))[0] > 0:
            count += 1

print(count)
        

#print(pil.getpixel((0,0)))
#print(pil.getpixel((0,5)))
#print(pil.getextrema())



tensor([[0, 0, 0, 0]])
(255, 0, 0)
3


In [2]:
tensor_max_example = torch.tensor(
[
  [
      [1, 1, 1],
      [2, 2, 2],
      [3, 3, 3]
  ],
  [
      [4, 4, 4],
      [5,5, 5],
      [6, 6, 6]
  ]
]
)

tensor_max_value = torch.max(tensor_max_example)
print(tensor_max_value)


tensor(6)


In [11]:
!python3 ./test.py --image_dir ./data/validation/ --model_path ./models/resnet16.pt --labels_file=./data/labels/labels.txt --visual_dir ./visual

Namespace(image_dir='./data/validation/', image_path='./data/test/120.jpg', labels_file='./data/labels/labels.txt', model_path='./models/resnet16.pt', visual_dir='./visual')
./data/validation/115.jpg
0.760230 0.321659
Testing error: 0.011718
./data/validation/112.jpg
0.422838 0.831939
Testing error: 0.013060
./data/validation/118.jpg
0.529651 0.479116
Testing error: 0.008648
./data/validation/111.jpg
0.447512 0.699551
Testing error: 0.023403
./data/validation/119.jpg
0.479573 0.441656
Testing error: 0.010124
./data/validation/113.jpg
0.509417 0.490862
Testing error: 0.011440
./data/validation/116.jpg
0.445802 0.765849
Testing error: 0.026075
./data/validation/114.jpg
0.789693 0.682633
Testing error: 0.011342
./data/validation/117.jpg
0.642867 0.204300
Testing error: 0.020537
./data/validation/125.jpg
0.817054 0.710102
Testing error: 0.018113
